# Holiday and Fraud analysis

In [17]:
import datetime
import re
from pyspark.sql.functions import *
from pyspark.sql.types import DateType, FloatType
from pyspark.sql import SparkSession, functions as F


In [3]:
# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("Data_Explorer")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.driver.memory", "4g")
    .getOrCreate()
)

22/10/06 01:03:59 WARN Utils: Your hostname, Loky-PC resolves to a loopback address: 127.0.1.1; using 192.168.55.225 instead (on interface eth0)
22/10/06 01:03:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/06 01:04:01 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [60]:
import pandas as pd
data = ['2021-03-08','2021-04-02','2021-04-03','2021-04-04','2021-04-05','2021-04-25','2021-06-14','2021-09-24','2021-11-02','2021-12-25','2021-12-26','2021-12-27','2021-12-28',
       '2022-01-01','2022-01-03','2022-01-26','2022-03-14','2022-04-15','2022-04-16','2022-04-17','2022-04-18','2022-04-25',
       '2022-06-13','2022-09-22','2022-09-23']
dates = pd.date_range(start='2021-02-01', end='2022-10-31')

list1 = []
for date in dates:
    if date in data:
        list1.append((date.strftime('%Y-%m-%d'), True))
    else:
        list1.append((date.strftime('%Y-%m-%d'), False))
holiday_data = spark.createDataFrame(pd.DataFrame(list1, columns =['Date', 'holiday?']))
holiday_data = holiday_data.withColumn("Date",col("Date").cast(DateType()))



In [101]:
holiday_pd = holiday_data.toPandas()

consumer_fraud = pd.read_csv("../data/tables/consumer_fraud_probability.csv")
consumer_fraud["date"] = pd.to_datetime(consumer_fraud["order_datetime"])
consumer_fraud = consumer_fraud.drop(columns="order_datetime")
fraud_by_date = consumer_fraud.groupby("date").mean("fraud_probablity")
holiday_pd["date"] = pd.to_datetime(holiday_pd["Date"])
holiday_pd = holiday_pd.drop(columns="Date")

fraud_holiday = holiday_pd.join(fraud_by_date, on="date")
fraud_holiday.groupby("holiday?").mean("fraud_probablity").head()




,user_id,fraud_probability
holiday?,,
False,12002.025951,14.903774


In [130]:
holiday_data.printSchema()
consumer_fraud = spark.read.option("header", True).csv("../data/tables/consumer_fraud_probability.csv")

consumer_fraud = consumer_fraud.withColumn("Date", col("order_datetime").cast(DateType()))
consumer_fraud = consumer_fraud.withColumn("f_p", col("fraud_probability").cast(FloatType()))

consumer_fraud = consumer_fraud.groupBy("Date").mean("f_p")

joined = consumer_fraud.join(holiday_data, on = "Date")
joined.show(5)
a = joined.groupBy("holiday?").mean("avg(f_p)")
a.show(5)







root
 |-- Date: date (nullable = true)
 |-- holiday?: boolean (nullable = true)

+----------+------------------+--------+
|      Date|          avg(f_p)|holiday?|
+----------+------------------+--------+
|2021-02-28|15.891153547498915|   false|
|2021-03-01|12.363714536031088|   false|
|2021-03-02|16.292744318644207|   false|
|2021-03-03|14.011275291442871|   false|
|2021-03-04|15.537732404821059|   false|
+----------+------------------+--------+
only showing top 5 rows

+--------+------------------+
|holiday?|     avg(avg(f_p))|
+--------+------------------+
|   false|14.903774021734295|
+--------+------------------+

